<a href="https://colab.research.google.com/github/senasung37/recommendation/blob/main/movie_recommender_surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코드출처: 이수안 컴퓨터 연구소 <br>
- 링크: https://www.youtube.com/watch?v=6TP51jvjLsE&t=698s

#라이브러리 & 데이터 불러오기

surprise: 추천 시스템 개발을 위한 라이브러리 

In [2]:
#surprise 라이브러리 설치
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 7.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633961 sha256=7411b1c6ffd7808f27c1c5c592ddf0d6b723b35c221ba9b7392565de23a58285
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
#라이브러리 불러오기
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [4]:
#데이터 불러오기 (ml-100k: 데이터 종류)
#피쳐: 사용자id, 영화id, 평점, 시간)
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [5]:
#모델 설정(SVD - matrix factorization 알고리즘, 비어있는 값들 예측)
model = SVD()

In [6]:
#모델 검증
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9344  0.9386  0.9333  0.9416  0.9379  0.9371  0.0030  
MAE (testset)     0.7353  0.7418  0.7358  0.7412  0.7394  0.7387  0.0027  
Fit time          5.66    9.22    5.29    5.20    4.66    6.01    1.64    
Test time         0.58    0.23    0.25    0.24    0.12    0.28    0.15    


{'fit_time': (5.6599507331848145,
  9.222877979278564,
  5.292036533355713,
  5.1974196434021,
  4.656642198562622),
 'test_mae': array([0.73530231, 0.74177846, 0.73579188, 0.74119352, 0.73940918]),
 'test_rmse': array([0.93437325, 0.93855632, 0.93333692, 0.94155208, 0.93787465]),
 'test_time': (0.5772707462310791,
  0.22705674171447754,
  0.2518765926361084,
  0.24018383026123047,
  0.12134313583374023)}

#컨텐츠 기반 필터링
사용자가 좋아하는 상품과 유사한 상품 추천(예시: 나와 비슷한 취향의 사용자가 시청한 영화 추천)
- 장점: 많은 수의 사용자 대상으로 쉽게 확장, 관심 없던 상품 추천 가능
- 단점: 입력 특성을 직접 설계해야 돼서 도메인 지식 필요, 사용자 기존 관심사항 기반으로만 추천 가능

## dot product를 사용한 추천

In [7]:
import numpy as np
from surprise import Dataset

In [8]:
#데이터 array로 만들기
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [9]:
#userid, movieid가 0부터 시작하도록 변경
raw_data[:,0] -= 1
raw_data[:,1] -= 1
raw_data


array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       ...,
       [      275,      1089,         1, 874795795],
       [       12,       224,         2, 882399156],
       [       11,       202,         3, 879959583]])

In [10]:
#유저, 영화 최대값으로 shape 만들기
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [11]:
#user x movie 인접행렬 만들기(데이터가 있으면 1, 없으면 0)
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id, movie_id] = 1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [12]:
#dot product 유사도 평가 알고리즘 - 입력한 id와 가장 유사한 similarity 및 ID 도출
my_id = int(input())
my_vector = adj_matrix[my_id]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match Similarity: {}, Best Match ID: {}".format(best_match, best_match_id))

0
Best Match Similarity: 183, Best Match ID: 275


In [18]:
#추천리스트 뽑기(내가 보지 않았지만 유사 ID가 본 영화)
recommend_lst = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_lst.append(i)
print(recommend_lst)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

## 유클리드 거리를 사용한 추천

In [26]:
#유클리드 거리 유사도 평가 알고리즘
my_id = int(input())
my_vector = adj_matrix[my_id]
best_match, best_match_id, best_match_vector = 999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euc_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euc_dist < best_match:
      best_match = euc_dist
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match Euclidean Distance: {}, Best Match ID: {}".format(best_match, best_match_id))

0
Best Match Euclidean Distance: 14.832396974191326, Best Match ID: 737


In [29]:
#추천리스트 뽑기(내가 보지 않았지만 유사 ID가 본 영화)
recommend_lst = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_lst.append(i)
print(recommend_lst)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


## 코사인 유사도를 사용한 추천

In [32]:
#코사인 유사도 함수
def compute_cos_sim(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)

In [34]:
#코사인 유사도 평가 알고리즘
my_id = int(input())
my_vector = adj_matrix[my_id]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_sim = compute_cos_sim(my_vector, user_vector)
    if cos_sim > best_match:
      best_match = cos_sim
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match Euclidean Distance: {}, Best Match ID: {}".format(best_match, best_match_id))

0
Best Match Euclidean Distance: 0.5278586163659506, Best Match ID: 915


In [35]:
#추천리스트 뽑기(내가 보지 않았지만 유사 ID가 본 영화)
recommend_lst = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_lst.append(i)
print(recommend_lst)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


## 평점을 활용한 추천

In [36]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id, movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [39]:
#유클리드 거리 유사도 평가 알고리즘
my_id = int(input())
my_vector = adj_matrix[my_id]
best_match, best_match_id, best_match_vector = 999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    euc_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euc_dist < best_match:
      best_match = euc_dist
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match Euclidean Distance: {}, Best Match ID: {}".format(best_match, best_match_id))

0
Best Match Euclidean Distance: 55.06359959174482, Best Match ID: 737


In [40]:
#코사인 유사도 평가 알고리즘
my_id = int(input())
my_vector = adj_matrix[my_id]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_sim = compute_cos_sim(my_vector, user_vector)
    if cos_sim > best_match:
      best_match = cos_sim
      best_match_id = user_id
      best_match_vector = user_vector
      
print("Best Match Euclidean Distance: {}, Best Match ID: {}".format(best_match, best_match_id))

0
Best Match Euclidean Distance: 0.569065731527988, Best Match ID: 915
